# **Pre Training of BERT Model using Mask Language Modeling and Next Sentence Prediction**

In [2]:
!pip install transformers datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


## Here we are importing only architecture of bert-base-uncased without taking trained weights.

In [ ]:

from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',bos_token = "<|BOS|>")
from transformers import BertForPreTraining, BertConfig

# Load the configuration for the model
config = BertConfig.from_pretrained('bert-base-uncased')
# Create a new instance of the model using the configuration
model = BertForPreTraining(config)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()
from datasets import load_dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
# config = BertConfig.from_pretrained('bert-base-uncased')

# # Create a new instance of the model using the configuration
# model = BertForPreTraining(config)
# k=sum(p.numel() for p in model.parameters() if p.requires_grad)
# k

## Storing training data into train.txt and testing data into test.txt

In [ ]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(dataset["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(dataset["test"], "test.txt")

In [ ]:
with open('./train.txt', 'r') as fp:
    text = fp.read().split('\n')


In [ ]:
for element in text:
    if "=" in element:
        text.remove(element)
text = list(filter(lambda x: x != "", text))


In [ ]:
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)
bag_size

100212

# Created dataset for Next sentence prediction

In [ ]:
#Here we are creating two list sentence_a and sentence_b.
#setence_b contains next sentence corresponding to sentence in sentence_a.
# 50% of time , next sentence will be actaul next sentence and 50% of time it will be random sentence.

import random

sentence_a = []
sentence_b = []
next_sentence_label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            next_sentence_label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            next_sentence_label.append(1)

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_text, input_mask, segment_ids, target):
        self.input_text = input_text
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.target = target

In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

# Adding [CLS],[SEP] tokens in training text before tokenization.

In [ ]:
def convert_sentence_pair(titles, descs, max_seq_length, tokenizer):
    features = []
    for (ex_index, (title, desc)) in enumerate(zip(titles, descs)):
        tokens_a = tokenizer.tokenize(title)
        tokens_b = None
        tokens_b = tokenizer.tokenize(desc)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)
        input_text = (tokens)
        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_text)
        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_text))
        input_mask += padding
        segment_ids += padding
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        features.append(
                InputFeatures(
                    input_text=input_text,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    target=1
        ))
    return features


In [ ]:
correct_pairs = convert_sentence_pair(sentence_a, sentence_b, max_seq_length=128, tokenizer=tokenizer)


In [ ]:
text=[]
for i in range(len(correct_pairs)):
   text.append(" ".join(correct_pairs[i].input_text[1:]))
#text = text[:5]

# Tokenize the training data

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True, padding=True)
inputs

{'input_ids': tensor([[  101,  2207,  1999,  ...,     0,     0,     0],
        [  101,  1037,  2312,  ...,     0,     0,     0],
        [  101,  2009,  2001,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  2691,  ...,     0,     0,     0],
        [  101, 13177,  2018,  ...,     0,     0,     0],
        [  101,  2691,  2732,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
#For MLM, We are storing actual tokens into inputs dictionary with key 'labels'.
inputs['labels'] = inputs.input_ids.detach().clone().to(device)

In [ ]:
#For NSP, We are storing 0,1 values correspoinding to each pair.
#  0 indicates next sentence is actual next sentence
# 1 indicates next sentence is random sentence of corpus.
inputs['next_sentence_label'] = torch.tensor(next_sentence_label).to(device)

In [ ]:
inputs.keys()
inputs['next_sentence_label']


tensor([1, 0, 1,  ..., 0, 0, 1], device='cuda:0')

# For MLM, We are masking 15% of tokens and among them 80% are masked with [MASK], 10% are masked with random word token and 10% are kept as it is.

In [ ]:
#Here we mask tokens with [MASK] token
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.12) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)
selection=[]
for i in range(mask_arr.shape[0]):
  selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:

rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.015) * (inputs.input_ids != 101)*(inputs.input_ids != 102) * (inputs.input_ids != 0)*(inputs.input_ids != 103)
selection=[]
for i in range(mask_arr.shape[0]):
  selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())



In [ ]:
# Here we mask token with random token
import random
for i in range(inputs.input_ids.shape[0]):
  for j in range(len(selection[i])):
    random_id = random.randrange(0,tokenizer.vocab_size)
    inputs.input_ids[i,selection[i][j]] = random_id


In [ ]:

class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
#create dataset for training
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

# Used ADAM optimizer with learning rate 5e-5

In [ ]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
data = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")["text"]
data = [s for s in data if (s!='' and len(s.split(' ')) < 300 and len(s.split(' ')) > 20)]



In [ ]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


# Train model with 5 epochs and finds perplexity at every epoch.

In [ ]:

from tqdm import tqdm  # for our progress bar
from math import exp
import evaluate
epochs = 5

total_loss = 0
count = 0
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    total_loss = 0
    count = 0
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)

        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels,next_sentence_label=next_sentence_label)
        # extract loss
        loss = outputs.loss.to(device)
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        total_loss += loss.item()
        count += 1
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    model.save_pretrained('my_model.h5')
    tokenizer.save_pretrained('my_model.h5')
    perplexity = evaluate.load("perplexity", module_type="metric")
    results = perplexity.compute(model_id='my_model.h5',
                                predictions=data,
                                add_start_token=False)
    print("Perplexity: ",round(results["mean_perplexity"], 2))


  0%|          | 0/981 [00:00<?, ?it/s]<ipython-input-21-6dccb434beb5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 981/981 [06:58<00:00,  2.34it/s, loss=4.89]
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


  0%|          | 0/113 [00:00<?, ?it/s]

Perplexity:  10726.49


Epoch 1: 100%|██████████| 981/981 [06:57<00:00,  2.35it/s, loss=4.29]
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/113 [00:00<?, ?it/s]

Perplexity:  15030.8


Epoch 4: 100%|██████████| 981/981 [06:57<00:00,  2.35it/s, loss=4.15]
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


  0%|          | 0/113 [00:00<?, ?it/s]

Perplexity:  17430.77


# Save pre trained model in colab session.

In [ ]:
model.save_pretrained('my_model.h5')


# Pushed pre trained model on huggingface hub

In [ ]:

pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("Bhautiksinh/Bert_Pretrain_Final")
tokenizer.push_to_hub("Bhautiksinh/Bert_Pretrain_Final")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bhautiksinh/Bert_Pretrain_Final/commit/424285e268536ee77da05cd0bdf0ef89dc276e2e', commit_message='Upload tokenizer', commit_description='', oid='424285e268536ee77da05cd0bdf0ef89dc276e2e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("Bhautiksinh/Bert_Pretrain_Model")

CommitInfo(commit_url='https://huggingface.co/Bhautiksinh/Bert_Pretrain_Model/commit/4c86a8bab2ddcedb0e53844003165416a7b7ac37', commit_message='Upload tokenizer', commit_description='', oid='4c86a8bab2ddcedb0e53844003165416a7b7ac37', pr_url=None, pr_revision=None, pr_num=None)

# Code for Calculating Parameters

In [7]:
from transformers import BertForPreTraining, BertConfig,AutoModelForSequenceClassification,AutoModel
num_classes = 5
model = AutoModel.from_pretrained("Bhautiksinh/BertPretrain", num_labels=num_classes)
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Assuming your fine-tuned BERT model is named 'model'
model = model.to(device)  # Move the model to the desired device
params={}
def print_parameters(model):
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_params = param.numel()
            params[name]=param.numel()
            total_params += num_params
            print(f"{name}: {num_params}")
    #print(params)
    print(f"Total Parameters: {total_params}")

# Print all parameter names and sizes
print_parameters(model)


embeddings.word_embeddings.weight: 23440896
embeddings.position_embeddings.weight: 393216
embeddings.token_type_embeddings.weight: 1536
embeddings.LayerNorm.weight: 768
embeddings.LayerNorm.bias: 768
encoder.layer.0.attention.self.query.weight: 589824
encoder.layer.0.attention.self.query.bias: 768
encoder.layer.0.attention.self.key.weight: 589824
encoder.layer.0.attention.self.key.bias: 768
encoder.layer.0.attention.self.value.weight: 589824
encoder.layer.0.attention.self.value.bias: 768
encoder.layer.0.attention.output.dense.weight: 589824
encoder.layer.0.attention.output.dense.bias: 768
encoder.layer.0.attention.output.LayerNorm.weight: 768
encoder.layer.0.attention.output.LayerNorm.bias: 768
encoder.layer.0.intermediate.dense.weight: 2359296
encoder.layer.0.intermediate.dense.bias: 3072
encoder.layer.0.output.dense.weight: 2359296
encoder.layer.0.output.dense.bias: 768
encoder.layer.0.output.LayerNorm.weight: 768
encoder.layer.0.output.LayerNorm.bias: 768
encoder.layer.1.attention.s